```
from pathlib import Path

folder_path = Path("your/folder/path")
csv_files = [f for f in folder_path.glob("*.csv") if f.is_file() and f.stat().st_size > 0]

print(csv_files)
```

In [2]:
import logging

from collections import defaultdict
from csv import DictReader, reader
from datetime import datetime
from pathlib import Path
from prettytable import PrettyTable, TableStyle
from typing import Dict, List

In [ ]:
class Schedule:
	def __init__(self, employees: dict, template: dict, weekday: str):
		self.employees = employees
		self.template = template
		self.weekday = weekday
		self.hours = {
			"weekday": ["9-11", "11-1", "1-2", "2-4", "4-6", "6-8"],
			"weekend": ["9-11", "11-12", "12-1", "1-2", "2-4", "4-6"],
			"sunday": ["2-3", "3-4", "4-5", "5-6"]
		}
		self.time_slots = {}
		self.locations = {}
		self.time_length = 0

		self._generate_time_slots()

		self.ft_employees, self.ft_hours = self._get_workers_by_position("full time")
		self.pt_employees, self.pt_hours = self._get_workers_by_position("part time")
		self.sc_employees, self.sc_hours = self._get_workers_by_position("security full time")
		sc_e_pt, sc_h_pt = self._get_workers_by_position("security part time")
		self.sc_employees += sc_e_pt; self.sc_hours += sc_h_pt

		string_header = {}
		for employee in self.employees.values():
			if self.weekday + "-hours" in employee.hours and employee.hours[self.weekday + "-hours"] and employee.position == "full time":
				

		self.content = {
			"times": [""] + list(self.time_slots.keys()),
			"puw": ["pick-up window"] + [""] * self.time_length,
			"fl": ["floor lead"] + [""] * self.time_length,
			"sp1a": ["service point 1"] + [""] * self.time_length,
			"sp1b": ["service point 1"] + [""] * self.time_length,
			"sp2a": ["service point 2"] + [""] * self.time_length,
			"sp2b": ["service point 2"] + [""] * self.time_length
		}
		
		self.header_table = PrettyTable(["who works today", "lunch breaks", "schedule changes"])

		self.content_table = None
	
	def _generate_time_slots(self):
		if self.weekday in ["monday", "tuesday", "wednesday", "thursday"]:
			for h in self.hours["weekday"]:
				self.time_slots[h] = []
				self.time_slots[h] = [datetime.strptime(h.split("-")[0].zfill(2) + "00", "%H%M").time(), datetime.strptime(h.split("-")[1].zfill(2) + "00", "%H%M").time()]
		elif "friday" in self.weekday or "saturday" in self.weekday:
			for h in self.hours["weekend"]:
				self.time_slots[h] = []
				self.time_slots[h] = [datetime.strptime(h.split("-")[0].zfill(2) + "00", "%H%M").time(), datetime.strptime(h.split("-")[1].zfill(2) + "00", "%H%M").time()]
		elif "sunday" in self.weekday:
			for h in self.hours["sunday"]:
				self.time_slots[h] = []
				self.time_slots[h] = [datetime.strptime(h.split("-")[0].zfill(2) + "00", "%H%M").time(), datetime.strptime(h.split("-")[1].zfill(2) + "00", "%H%M").time()]
		self.time_length = len(self.time_slots.keys())
	
	def _get_workers_by_position(self, position: str):
		working_employees = []
		hours = []
		for employee in self.employees.values():
			if self.weekday + "-hours" in employee.hours and employee.hours[self.weekday + "-hours"] and employee.position == position:
				working_employees.append(employee)
				if employee.hours[self.weekday + "-hours"] not in hours:
					hours.append([h for h in employee.hours[self.weekday + "-hours"]])
		return (working_employees, hours)

In [4]:
class Template:
	def __init__(self, location: Dict[str, List[str]]):

		self.location = location

```csv
puw,fl,sp1a,sp1b,sp2a,sp2b
eric,yvette,wilbur,sean,chris,jasper
wilbur,none,samuel,eric/1200,sean,corey/1200
antwon,none,corey,eric,rob,none
chris/1530/christa,none,lee,none,antwon,none
sean/1730/saxon,jordan,christa,derrick/1630,wanda,eric/1730
jordan,lee,wanda,saxon,derrick,christa
```

In [5]:
class Employee:

	def __init__(
		self,
		name: str,
		rank: int,
		position: str,
		hours: Dict[str, List[str]],
		lunch_hours: Dict[str, List[str]],
	):

		self.name = name
		self.first_name = self._extract_first_name(name)
		self.initials = self._generate_initials(name)

		self.rank = rank
		self.position = position

		self.hours = self._convert_to_datetime(hours)
		self.phours = self._convert_to_ptime(hours)
		self.lunch_hours = self._convert_to_datetime(lunch_hours)
		self.plunch_hours = self._convert_to_ptime(lunch_hours)

		self.programs = None

	def _extract_first_name(self, full_name: str) -> str:

		return full_name.split()[0]

	def _generate_initials(self, name: str) -> str:

		return "".join(i[0] for i in name.replace("-", " ").split())

	def _convert_to_datetime(self, hours):

		result = {}
		for day, times in hours.items():
			result[day] = [datetime.strptime(t.zfill(4), "%H%M").time() for t in times]
		return result

	def _convert_to_ptime(self, hours):

		result = {}
		for day, times in hours.items():
			dt = [datetime.strptime(t.zfill(4), "%H%M") for t in times]
			hours = [str(t.hour % 12) or "12" for t in dt]
			mins = ["" if t.minute == 0 else str(t.minute) for t in dt]
			ft = [":".join([str(h), f"{m}"]).rstrip(":") for h, m in zip(hours, mins)]
			result[day] = ft
		return result

	def _get_schedule_summary(self) -> Dict[str, int]:

		return {
			"work_days": len(self.hours.keys()),
			"lunch_days": len(self.lunch_hours.keys()),
		}

	def __str__(self) -> str:

		name_info = f"Abbv: {self.initials}\t || First: {self.first_name}"
		pos_info = f"Rank: {self.rank:>2}\t || Position: {self.position}"

		schedule_summary = self._get_schedule_summary()
		schedule_info = f"Hrs: {schedule_summary['work_days']:>3}\t || Lunch hrs: {schedule_summary['lunch_days']}"

		lines = [name_info, pos_info, schedule_info]
		max_width = max(len(line.expandtabs()) for line in lines)

		name_padding = max(0, (max_width - len(self.name)) // 2)
		centered_name = " " * name_padding + self.name

		divider = "=" * max_width

		formatted_sections = [
			divider,
			centered_name,
			divider,
			name_info,
			pos_info,
			schedule_info,
		]

		return "\n".join(formatted_sections) + "\n"

In [6]:
class Loader:
	
	def __init__(self):
		self.employees = {}
		self.templates = {}
		
		self.key = None
		self.locations_key = {
			"puw": "pick-up window",
			"fl": "floor lead",
			"sp1a": "service point 1",
			"sp1b": "service point 1",
			"sp2a": "service point 2",
			"sp2b": "service point 2"
		}
	
	def load_employees(self, employees_path: str, employees_key: str = ""):
		try:
			key_map = {}
			with open(employees_key, "r", encoding="utf-8") as keyfile:
				for line in keyfile:
					encoded, decoded = line.split("= ")
					key_map[encoded.strip()] = decoded.strip()
			self.key = key_map
		except FileNotFoundError as e:
			logging.exception(f"Path: '{employees_key}' does not exist. {e}")
			self.key = None
		
		employees = {}
		try:
			fields = []
			rows = []
			with open(employees_path, "r", encoding="utf-8") as csvfile:
				csvreader = reader(csvfile)
				fields = next(csvreader)
				for row in csvreader:
					rows.append(row)
			
			for row in rows:
				name = self.key.get(row[0], row[0]) if self.key else row[0]
				rank, position = row[1], row[2]
				hours, lunch_hours = {}, {}
				
				for i in range(3, len(fields)):
					key, value = fields[i], row[i]
					if "-hours" in key and value != "off":
						hours[key] = value.split("-")
					if "-lunch" in key and value != "none":
						lunch_hours[key] = value.split("-")
				
				employee = Employee(name, rank, position, hours, lunch_hours)
				employees[employee.first_name] = employee
			
			return employees
			
		except FileNotFoundError as e:
			logging.exception(f"Path: '{employees_path}' does not exist. {e}")
		except Exception as e:
			logging.exception(f"Failed to load '{employees_path}': {e}")
		
		return employees

	def load_templates(self, templates_path: str):
		templates_dir = Path(templates_path)
		csv_files = [str(f).replace("\\", "/") for f in templates_dir.glob("*.csv") if f.is_file() and f.stat().st_size > 0]

		if len(csv_files) > 0:
			# self.templates = [self._load_template(f) for f in csv_files]
			for f in csv_files:
				template_day = Path(f).stem.replace("_template", "")
				self.templates[template_day] = self._load_template(f)
			
			return self.templates


	def _load_template(self, template_path: str):
		data = defaultdict(list)
		try:
			fields = []
			rows = []
			with open(template_path, "r", encoding="utf-8") as csvfile:
				csvreader = DictReader(csvfile)
				for row in csvreader:
					for key, value in row.items():
						key = self.locations_key[key]
						working_employees = []
						if self.key:
							if "/" in value:
								raw_pairs = value.split("/")
								employee_pairs = [self._get_decoded_first_name(e) for e in raw_pairs]
								if employee_pairs[0].isdigit():
									employee_pairs[0] = self._convert_to_datetime(employee_pairs[0])
								elif employee_pairs[1].isdigit():
									employee_pairs[1] = self._convert_to_datetime(employee_pairs[1])
								data[key].append(employee_pairs)
							else:
								data[key].append(self._get_decoded_first_name(value))
			
			result = dict(data)
			return result
		
		except FileNotFoundError as e:
			logging.exception(f"Path: '{template_path}' does not exist. {e}")
		except Exception as e:
			logging.exception(f"Failed to load '{template_path}': {e}")
	
	def _get_decoded_first_name(self, query: str) -> str:
		query = query.lower()
		if self.key:
			for k, v in self.key.items():
				first_name = k.split()[0].lower()
				if first_name == query:
					return v
		return query if query != "none" else ""

	def _convert_to_datetime(self, hour: str):
		return datetime.strptime(hour.zfill(4), "%H%M").time()

In [111]:
loader = Loader()

employees = loader.load_employees("../models/employees.csv", "../models/employees_key.txt")
templates = loader.load_templates("../models/templates")

tuesday = Schedule(employees, templates["tuesday"], "tuesday")

['0900', '0900', '1400']


In [ ]:
def _get_workers_by_position(self, position: str):
	working_employees = []
	hours = []
	for employee in self.employees.values():
		if self.weekday + "-hours" in employee.hours and employee.hours[self.weekday + "-hours"] and employee.position == position:
			working_employees.append(employee)
			if employee.hours[self.weekday + "-hours"] not in hours:
				hours.append([h for h in employee.hours[self.weekday + "-hours"]])
	return (working_employees, hours)